In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame()

# reading in each of our ten files
for i in range(1, 11):
    filename = f"0001-Data-{i}.tsv"
    
    # reading visit file into a dataframe
    with open(filename, "r") as infile:
        visit_data = pd.read_csv(infile, sep = "\t")
        
        # creating a column for the visit number
        visit_data["visit"] = i
        
        # removing the visit number from the end of each attribute name
        for col in visit_data.columns:
            if col[-len(str(i)):] == str(i):
                visit_data.rename(columns={col:col[:-len(str(i))]}, inplace=True)
        
        # adding this visit data back to the merged dataframe
        data = pd.concat([data, visit_data], ignore_index = True)

/Users/victoriamccray/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/victoriamccray/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,51,52,53,54,57,58,59,60,61,63,64,65,66,67,68,69,70,71,72,73,74,75,85,87,88,90,91,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,477,478,482,483,484,485,486,490,491,495,504,506,507,515,522,530,536,537,538,539,577) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/victoriamccray/opt/anaconda3/lib/python3.8/site-packages/IPython/core/

In [2]:
# replacing blank cells with NaN
data = data.replace(r'^\s*$', np.nan, regex=True)

for col in data.columns:
    # removing columns about family age
    if "AGE" in col and col != "AGE":
        del data[col]
    
    # removing columns with > 75% NaN values
    if col in data:
        nas = data[col].isna().sum()
        if nas > 3 * len(data[col]) / 4:
            del data[col]


In [3]:
# TODO implement lasso with all features of the data, 
# remove all null rows or estimate values because the model won't worrk with null rows

In [5]:
# test features with first columns
col_list = list(data.columns[:7])

In [6]:
# test x feat with just int columns
x = data.loc[:, col_list]
# drop null rows
x.dropna(axis=0, how='any', inplace=True)


# age and id are columns for y df
y = pd.DataFrame()

y['AGE'] = x['AGE']
y['SWANID'] = x['SWANID']

# remove predicted feature
x.drop(['AGE'], axis=1, inplace=True)


In [7]:

# check for null values by col
pd.isnull(x).sum()

SWANID     0
VISIT      0
INTDAY     0
LANGINT    0
RACE       0
PREGNAN    0
dtype: int64

In [8]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [9]:
# split data by train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=0)

In [10]:
# Lasso
from sklearn.linear_model import Lasso

alpha = 0.1
lasso = Lasso(alpha=alpha)

# fit lasso to data
y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
r2_score_lasso = r2_score(y_test, y_pred_lasso)

print(lasso)
print("r^2 on test data : %f" % r2_score_lasso)


Lasso(alpha=0.1)
r^2 on test data : 0.766279
